# SDialog dependencies

In [1]:
import os
import json
from tqdm import tqdm

In [2]:
import sdialog
from sdialog import Dialog
from sdialog.generators import PersonaGenerator
from sdialog.personas import Persona, PersonaAgent, Doctor, Patient, Agent

/Users/yanislabrak/opt/miniconda3/envs/flask/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Generate Persona

In [ ]:
persona_card_folder = "./personas"

sdialog.config.set_llm("aws:anthropic.claude-3-5-sonnet-20240620-v1:0", region_name="us-east-1")

# Generate doctor persona
doctor_persona = Doctor(
    name="Dr. Smith",
    gender="male",
    age=52,
    specialty="Family Medicine"
)
generator_doctor = PersonaGenerator(doctor_persona)
persona_cards = generator_doctor.generate(n=1)
persona_cards.to_file(f"{persona_card_folder}/persona_doctor.json")

# Generate patient persona
patient_persona = Patient(
    name="John Doe",
    gender="male",
    age=62
)
generator_patient = PersonaGenerator(patient_persona)
persona_cards = generator_patient.generate(n=1)
persona_cards.to_file(f"{persona_card_folder}/persona_patient.json")


# Load persona

In [3]:
# Load personas
persona_doctor = Persona.from_file("./personas/persona_doctor.json")
persona_patient = Persona.from_file("./personas/persona_patient.json")

In [ ]:
context = "Generate me a 50 turn medical dialogue between patient and doctor, for a primary care visit"

# Create agents
agent1 = PersonaAgent(persona=persona_doctor, name="DOCTOR", dialogue_details=context, response_details="make short turn answers when needed")
agent2 = PersonaAgent(persona=persona_patient, name="PATIENT", dialogue_details=context, response_details="make short turn answers when needed")

In [4]:
save_all = True
GENERATE_PERSONA = True
FORCE_DIALOG_GENERATION = False

os.makedirs("./outputs", exist_ok=True)

In [5]:
if FORCE_DIALOG_GENERATION:

    dialog = agent1.talk_with(agent2, max_turns=3)
    dialog.to_file("dialog_demo.json")

else:
    dialog = Dialog.from_file("dialog_demo.json")

dialog.print()

[dialog_id] 1752861530588
[model] client=<botocore.client.BedrockRuntime object at 0x7fc319ec91c0> model_id='anthropic.claude-3-5-sonnet-20240620-v1:0' region_name='us-east-1' provider='anthropic' supports_tool_choice_values=('auto', 'any', 'tool')
[seed] 226296126
--- Dialogue Begins ---
[DOCTOR] Hello there. I'm Dr. Smith. Welcome to my office. What brings you in to see me today?
[PATIENT] Good morning, Dr. Smith. I'm John Doe. I've been dealing with a persistent cough and feeling pretty tired for about three weeks now. It's starting to wear me down, and I thought I should get it checked out.
[DOCTOR] I'm sorry to hear you've been feeling unwell, Mr. Doe. A persistent cough and fatigue can certainly be troublesome. Let's get some more details. Can you describe your cough? Is it dry or productive?
[PATIENT] Well, Dr. Smith, it's mostly a dry cough. It's particularly bothersome at night when I'm trying to sleep. You know, it reminds me of the time I was teaching about the Civil War and

# Audio dependencies

# Instantiating voices database

In [6]:
from sdialog.audio.voice_database import DummyVoiceDatabase

In [7]:
dummy_voice_database = DummyVoiceDatabase()
print("Instantiating voice database...")

Instantiating voice database...


# Instantiating TTS pipeline

In [8]:
from sdialog.audio.tts_engine import KokoroTTS # ChatterboxTTS, XttsTTS

In [9]:
tts_pipeline = KokoroTTS()
# tts_pipeline = ChatterboxTTS()
# tts_pipeline = XttsTTS()
print("Instantiating TTS pipeline...")

/Users/yanislabrak/opt/miniconda3/envs/flask/lib/python3.9/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/Users/yanislabrak/opt/miniconda3/envs/flask/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Instantiating TTS pipeline...


In [10]:
from sdialog.audio.audio_dialog import AudioDialog

In [11]:
# Extend the turns with audio attributes
dialog: AudioDialog = AudioDialog.from_dialog(dialog)

In [12]:
from sdialog.audio import audio_pipeline

In [ ]:
dialog: AudioDialog = audio_pipeline(
    dialog,
    voice_database=dummy_voice_database,
    tts_pipeline=tts_pipeline,
    dir_audio="./outputs"
)

In [ ]:
from sdialog.audio.audio_events_enricher import AudioEventsEnricher

In [ ]:
# Enriching the dialog with audio events and generate the timeline of audio events and utterances
enricher = AudioEventsEnricher()
timeline = enricher.extract_events(dialog, dialog.turns)
timeline.print()

# Drawing the timeline
timeline.draw("./outputs/timeline.png")

In [ ]:
from sdialog.audio.evaluation import compute_evaluation_utterances, compute_evaluation_audio

In [ ]:
compute_evaluation_utterances(dialog)
compute_evaluation_audio(dialog)